In [1]:
# Installing pymupdf for the same 
%pip install PyMuPDF
%pip install pytesseract
%pip install opencv-python
%pip install numpy
import numpy as np 
import pytesseract
import cv2

In [31]:
#from google.colab import drive
#drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
import fitz
file = fitz.open('SROIE.pdf')

#Extracting the text
for PageNumber,Page in enumerate(file.pages(),start =1):

   text_data = Page.getText()
   text = open(f'Textfiles/report_pg_{PageNumber}.txt','a')
   text.writelines(text_data)
   text.close()


Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


In [2]:
from fitz.fitz import CS_RGB
#Extracting the images from the document
for PageNumber,Page in enumerate(file.pages(),start =1):
  for imgNumber, img in enumerate(Page.getImageList(),start =1):
    xref = img[0]
    pix = fitz.Pixmap(file,xref)
    # pix.n is the number of bytes per pixel and n>4 implies image is not grey or RGB image
    if pix.n>4:
      pix = fitz.Pixmap(fitz.CS_RGB, pix) 
    pix.writePNG(f'Training Images/image_Page{PageNumber}_{imgNumber}.png')


Deprecation: 'getImageList' removed from class 'Page' after v1.19 - use 'get_images'.
Deprecation: 'writePNG' removed from class 'Pixmap' after v1.19 - use 'save'.


In [2]:
#Reading the extracted Images from the dataset
from PIL import Image
import glob
image_list = []
i = 0
for filename in glob.glob('/content/Training Images/*.png'): #assuming png
  image=Image.open(filename)
  image_list.append(image)
  image = np.array(image)
  #image = cv2.imread("im.png")
  #base_image = image.copy()
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #cv2.imwrite(f"output/index_gray{i}.png", gray)
  blur = cv2.GaussianBlur(gray, (7,7), 0)
  #cv2.imwrite(f"output/index_blur{i}.png", blur)
  thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  #cv2.imwrite(f"output/index_thresh{i}.png", thresh)
  kernal = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 13))
  #cv2.imwrite(f"output/index_kernal{i}.png", kernal)
  dilate = cv2.dilate(thresh, kernal, iterations=1)
  #cv2.imwrite(f"output/index_dilate{i}.png", dilate)
  cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  # the condition for contour that we want to work
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])
  print(len(cnts))
  cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])
  #cv2.imwrite(f"output/index_bbox{i}.png", image)
  for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)
    print(x,y,w,h)
  #if h > 200 and w > 20:
    #   roi = image[y:y+h, x:x+h]
    #  cv2.imwrite("index_roi.png", roi)
      # cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)
      
    cv2.imwrite(f"output/index_bbox_all{i}.png", image)
  i+=1

173
0 1949 19 12
0 0 683 110
23 1951 7 10
33 187 3 13
34 1952 6 9
45 1953 6 8
45 520 4 13
46 763 4 15
47 440 4 13
56 1952 6 9
58 521 7 27
64 440 6 20
68 1952 9 9
73 559 7 14
73 540 3 14
77 680 3 13
77 641 6 14
79 1951 7 10
90 1952 7 9
90 641 4 13
93 696 5 15
98 1952 10 9
103 641 5 19
104 136 3 13
105 519 4 14
111 1079 4 13
112 1951 8 10
113 172 4 14
122 1950 35 11
122 1120 4 14
122 140 4 16
123 840 4 13
125 172 4 14
138 295 4 13
141 439 4 14
152 360 4 13
153 439 7 13
158 1952 7 9
166 360 5 13
167 321 7 14
168 439 6 13
170 1952 9 9
178 221 11 22
180 1951 8 10
184 324 3 14
189 171 3 13
190 1920 466 41
195 321 5 14
196 440 3 13
202 320 4 14
211 360 4 13
212 314 8 19
212 293 5 13
213 883 4 14
226 439 11 15
227 381 5 13
230 763 3 13
231 341 4 14
231 320 3 13
240 382 10 19
242 313 7 21
243 1557 3 14
245 1157 3 14
246 1516 5 13
248 170 4 13
255 303 4 14
257 919 8 15
257 152 16 31
258 781 3 13
259 761 5 14
260 439 4 13
262 1079 3 13
270 364 4 15
274 319 6 14
275 438 5 14
281 615 4 13
285 582 2

In [144]:

import re

# Read In Text
#Text_list = []
#i = 0
for filename in glob.glob('/content/Textfiles/*.txt'): #assuming txt
#fileName = "report_pg_5.txt"
  print(filename)
  pdfTextfile = open(filename, "r")
  pdfText = pdfTextfile.read()
# Split text into blocks separated by double line break.
  blocks = pdfText.split("\n")
  #print(blocks)
# Remove all new lines within blocks to remove arbitary line breaks
  blocks = map(lambda x : x.replace("\n", ""), blocks)
# Which blocks are figure captions?
  captions = []
  i=0
  count=0
  for block in blocks:
    if(i==1):
      captions.append(block)
      i=0
      break
    if re.search('^Fig. ', block, re.IGNORECASE):
      #print('block here', block)
      captions.append(block)
      i=1
    #print(count)
      count+=1
  #print(captions)
  length_captions=len(captions)
  if length_captions>0:
    ImgCaption= captions[0] + " " + captions[1]
    print(ImgCaption)
  else:
    print("No Image or caption")
  
    
  




/content/Textfiles/report_pg_3.txt
Fig. 2. Examples of scanned receipts for the competition tasks.
/content/Textfiles/report_pg_5.txt
Fig. 4. Top 10 methods for Task 2 - Scanned Receipt OCR.
/content/Textfiles/report_pg_6.txt
No Image or caption
/content/Textfiles/report_pg_4.txt
Fig. 3. Top 10 methods for Task 1 - Scanned Receipt Text Localisation.
/content/Textfiles/report_pg_2.txt
Fig. 1. Annotation format for the tasks.
/content/Textfiles/report_pg_1.txt
No Image or caption
